https://www.wired.com/story/best-wordle-tips/

In [ ]:
%load_ext kedro.ipython

In [ ]:
%reload_kedro

In [3]:
import sys
sys.path.append("../src/projectwordle")

In [4]:
import plotly.express as px
import plotly.graph_objects as go
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List
from IPython.display import HTML
from more_itertools import sliding_window, pairwise
from utils import (
    color_pattern_matching,
    plot_guess_stats,
    plot_challenge_stats,
    difficulty_distribution,
    plot_most_difficult_words,
    plot_guess_stats_highlighted_words
)

pl.Config(tbl_rows=50)

# Load Data

In [ ]:
five_letter_word_anagrams = catalog.load("five_letter_words_anagrams")
simulating_top_words = catalog.load("simulating_top_words")

In [6]:
simulating_top_words_tries_difficulty = (
    simulating_top_words
    .join(
        five_letter_word_anagrams.select("words", "anagrams", "anagram_num"),
        left_on="next_guess",
        right_on="words",
        how="left",
        coalesce=True
    )
    .select(
        "challenge", "guess", "letter_differences", "common_letters",
        "num_diff_letters", "num_common_letters", "num_matching_index",
        "match_pattern", "num_choices_after_guess", "possible_guesses",
        "challenge_in_possible_guesses", "next_guess", "group", "tries", "difficulty",
        "guess_word_anagrams", "anagram_num"
    )
)

In [7]:
simulating_top_words_tries_difficulty.head()

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8
"""gamed""","""paire""","""irp""","""ae""",3,2,1,"""BGBBY""",76,"""james, babel, label, wales, la…",false,"""james""",0,5,"""hard""","""paire, perai""",2
"""gamed""","""james""","""rpsij""","""aem""",5,3,3,"""BGGGB""",5,"""camel, cameo, mamey, gamed, ga…",true,"""camel""",0,5,"""hard""","""james""",1
"""gamed""","""camel""","""crpsilj""","""aem""",7,3,3,"""BGGGB""",3,"""mamey, gamed, gamey""",true,"""mamey""",0,5,"""hard""","""camel, clame, macle""",3
"""gamed""","""mamey""","""crpsilyj""","""aem""",8,3,3,"""YGGGB""",1,"""gamed""",true,"""gamed""",0,5,"""hard""","""mamey""",1
"""gamed""","""gamed""","""""","""aemgd""",0,5,5,"""GGGGG""",0,"""""",false,"""gamed""",0,5,"""hard""",null,null


In [8]:
simulating_top_words_tries_difficulty.height

3816960

In [9]:
(
    simulating_top_words_tries_difficulty
    .unique(subset=["group"], keep="first")
    ["difficulty"]
    .value_counts()
    .sort("count", descending=True)
)

difficulty,count
str,u32
"""moderate""",299943
"""hard""",264617
"""very hard""",60046
"""easy""",11554


# Create dataframe of first guesses

In [10]:
# Group by "Category" and return the first row of each group
first_guess = (
    simulating_top_words_tries_difficulty
    .group_by("group", maintain_order=True)
    .first()
)

# EDA

In [11]:
# Plot difficulty distribution
difficulty_distribution(
    simulating_top_words_tries_difficulty
)

## Most frequent number of guesses

In [12]:
(
    first_guess
    ["tries"]
    .value_counts()
    .sort(by="count", descending=True)
)

tries,count
u8,u32
4,203894
5,174322
3,96049
6,90295
null,60046
2,11486
1,68


## Top most popular next guess

In [13]:
# Top most popular next guess
top_ten_next_guess = (
    first_guess
    ["next_guess"]
    .value_counts()
    .sort(by="count", descending=True)
    .head(10)
)

In [14]:
top_ten_next_guess

next_guess,count
str,u32
"""sound""",12701
"""final""",9442
"""model""",9365
"""night""",9285
"""basic""",8349
"""shall""",7957
"""bring""",7805
"""china""",7246
"""south""",7136


In [15]:
# Plot difficulty distribution of challenge word based on top ten next guess
difficulty_distribution(
    simulating_top_words_tries_difficulty
    .filter(pl.col("next_guess").is_in(top_ten_next_guess["next_guess"]))
)

## Anagrams  

Only the most frequently occuring anagrams were only considered for root words after the five letter words were processed to remove plurals, names and places. As such there won't be any anagrams for `leads` as it's root word is `lead` which is a 4-letter word.  

However, if our challenge word is `leads`, or even its anagram `deals`, we can use their anagram `slade` to make a correct guess.

In [16]:
# where challenge and guess are anagrams, removing instances where you guess right
# the first time to see how quickly we converge on the challenge word

anagrams = (
    simulating_top_words_tries_difficulty
    .filter(
        (pl.col("letter_differences") == "")
        & (pl.col("num_common_letters") == 5)
        & (pl.col("challenge") != pl.col("guess"))
    )
)

anagrams.head()

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8
"""stela""","""least""","""""","""least""",0,5,0,"""YYYYY""",2,"""steal, stela""",true,"""steal""",5324,3,"""moderate""","""least, steal, slate, stale, st…",8
"""stela""","""steal""","""""","""least""",0,5,3,"""GGGYY""",1,"""stela""",true,"""stela""",5324,3,"""moderate""",null,null
"""stela""","""stale""","""""","""stale""",0,5,3,"""GGYGY""",1,"""stela""",true,"""stela""",5339,2,"""easy""",null,null
"""stela""","""salet""","""""","""salet""",0,5,1,"""GYYYY""",5,"""steal, slate, stale, stela, se…",true,"""steal""",5366,3,"""moderate""",null,null
"""stela""","""steal""","""""","""salet""",0,5,3,"""GGGYY""",1,"""stela""",true,"""stela""",5366,3,"""moderate""",null,null


In [17]:
# Display the DataFrame with colored patterns

anagrams_select_group = np.random.choice(anagrams["group"], 1, replace = False)[0]

HTML(
    simulating_top_words_tries_difficulty
    # .loc[lambda df_: df_["group"] == anagrams_select_group]
    .filter(pl.col("group") == anagrams_select_group)
    .to_pandas()
    .assign(
        match_pattern = lambda df_:
        color_pattern_matching(
            dataf=df_,
            challenge_col="challenge",
            guess_col="guess"
        )
    )
    [[
        "challenge", "guess", "match_pattern", "letter_differences",
        "common_letters", "next_guess", "num_choices_after_guess",
        "group", "tries", "difficulty", "guess_word_anagrams", "anagram_num"
    ]]
    .to_html(escape=False)
)


,challenge,guess,match_pattern,letter_differences,common_letters,next_guess,num_choices_after_guess,group,tries,difficulty,guess_word_anagrams,anagram_num
0,salet,slate,GYYYY,,slate,steal,4,144255,3,moderate,None,NaN
1,salet,steal,GYYYY,,slate,salet,1,144255,3,moderate,None,NaN
2,salet,salet,GGGGG,,slate,salet,0,144255,3,moderate,None,NaN
3,salet,salet,GGGGG,,slate,salet,0,144255,3,moderate,None,NaN
4,salet,salet,GGGGG,,slate,salet,0,144255,3,moderate,None,NaN
5,salet,salet,GGGGG,,slate,salet,0,144255,3,moderate,None,NaN


### Challenge words without `r` , `s` or `t`
These are some of the most frequently ocuuring letters. We can then see how our naive algorithm operates in coming to the correct guess.

In [18]:
challenge_words_without_r_s_t = (
    first_guess
    .filter(
        ~(pl.col("challenge").str.contains("r|s|t"))
        & ((pl.col("challenge") != (pl.col("guess"))))
    )
)

challenge_words_without_r_s_t.sample(n = 50, with_replacement = False).head()

group,challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,tries,difficulty,guess_word_anagrams,anagram_num
u32,str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u8,str,str,u8
78172,"""gulfy""","""morae""","""mroea""","""""",5,0,0,"""BBBBB""",724,"""night, willy, skill, fight, st…",false,"""night""",5,"""hard""","""morae""",1
572269,"""minke""","""stale""","""slat""","""e""",4,1,1,"""BBBBG""",413,"""judge, rogue, urine, curve, pr…",false,"""judge""",5,"""hard""",null,null
268372,"""pleon""","""saute""","""satu""","""e""",4,1,0,"""BBBBY""",723,"""field, riley, owner, enemy, or…",false,"""field""",6,"""hard""","""saute""",1
107727,"""bicep""","""sarge""","""sgar""","""e""",4,1,0,"""BBBBY""",560,"""model, beech, fleet, check, mo…",false,"""model""",4,"""moderate""","""sarge, sager, segar""",3
488652,"""adman""","""morae""","""ore""","""ma""",3,2,1,"""YBBGB""",40,"""human, assam, liman, amman, du…",false,"""human""",null,"""very hard""","""morae""",1


In [19]:
# Plot difficulty distribution of of challenge words that doesn't contain 'r', 's' or 't'
difficulty_distribution(challenge_words_without_r_s_t)

## Null values (Incomplete games)

In [20]:
incomplete_games = (
    simulating_top_words_tries_difficulty
    .filter(pl.col("tries").is_null())
)

In [21]:
incomplete_games.head()

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8
"""gamed""","""toise""","""siot""","""e""",4,1,0,"""BBBBY""",733,"""upper, needy, fever, break, le…",false,"""upper""",33,null,"""very hard""","""toise""",1
"""gamed""","""upper""","""prsiotu""","""e""",7,1,1,"""BBBGB""",109,"""level, hazel, angel, abbey, wh…",false,"""level""",33,null,"""very hard""","""upper""",1
"""gamed""","""level""","""vprsiotul""","""e""",9,1,1,"""BYBGB""",49,"""cheek, ached, abbey, naked, an…",false,"""cheek""",33,null,"""very hard""","""level""",1
"""gamed""","""cheek""","""vcprksihotul""","""e""",12,1,1,"""BBYGB""",25,"""abbey, annex, embed, waxen, fe…",false,"""abbey""",33,null,"""very hard""","""cheek, keech""",2
"""gamed""","""abbey""","""vbcprksihotuly""","""ea""",14,2,1,"""YBBGB""",5,"""waxen, zazen, ganef, gamed, da…",true,"""waxen""",33,null,"""very hard""","""abbey""",1


In [22]:
null_select_group = np.random.choice(incomplete_games["group"], 1, replace = False)[0]

HTML(
    incomplete_games
    .filter(pl.col("group") == null_select_group)
    .to_pandas()
    .assign(
        match_pattern = lambda df_:
        color_pattern_matching(
            dataf=df_,
            challenge_col="challenge",
            guess_col="guess"
        )
    )
    [[
        "challenge", "guess", "match_pattern",
        "letter_differences", "common_letters", "next_guess",
        "group", "tries", "difficulty", "guess_word_anagrams", "anagram_num"
    ]]
    .to_html(escape=False)
)

,challenge,guess,match_pattern,letter_differences,common_letters,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num
0,linny,share,BBBBB,rshea,,union,208004,NaN,very hard,"share, shear",2
1,linny,union,BYYBY,rsheoua,ni,windy,208004,NaN,very hard,union,1
2,linny,windy,BGGBG,wdrsheoua,niy,ginny,208004,NaN,very hard,windy,1
3,linny,ginny,BGGGG,wdrshgeoua,niy,tinny,208004,NaN,very hard,ginny,1
4,linny,tinny,BGGGG,wdrshgeotua,niy,finny,208004,NaN,very hard,tinny,1
5,linny,finny,BGGGG,wfdrshgeotua,niy,ninny,208004,NaN,very hard,finny,1


### Plot incomplete games stats

In [23]:
incomplete_games_stats = (
    incomplete_games
    .select("challenge")
    .unique(keep="first")
    .join(
        (
            incomplete_games
            .unique(subset="group", keep="first")
            ["challenge"]
            .value_counts()
        ),
        left_on="challenge",
        right_on="challenge"
    )
    .rename({"count": "incomplete_games_count"})
    .with_columns(
        pl.col("incomplete_games_count").cast(pl.UInt8),
        incomplete_games_pct = (
            (pl.col("incomplete_games_count") / first_guess["guess"].n_unique() * 100)
            .round(2)
            .cast(pl.Float32)
        )
    )
    .sort(by="incomplete_games_pct", descending=True)
)

incomplete_games_stats.head()

challenge,incomplete_games_count,incomplete_games_pct
str,u8,f32
"""laxer""",70,100.0
"""gager""",70,100.0
"""razer""",70,100.0
"""kerry""",70,100.0
"""zappy""",70,100.0


### Using `first_guess` dataframe

In [24]:
incomplete_games_count = (
    first_guess
    .filter(pl.col("difficulty") == "very hard")
    .group_by("challenge")
    .len()
    .rename({"len": "incomplete_games_count"})
    .sort("incomplete_games_count", descending=True)
)

incomplete_games_count.head()

challenge,incomplete_games_count
str,u32
"""razer""",70
"""gager""",70
"""zoner""",70
"""waxer""",70
"""koker""",70


In [25]:
plot_most_difficult_words(dataf=incomplete_games_stats)

## Words never guessed

In [26]:
(
    incomplete_games_stats
    .filter(pl.col("incomplete_games_pct") == 100)
)

challenge,incomplete_games_count,incomplete_games_pct
str,u8,f32
"""laxer""",70,100.0
"""gager""",70,100.0
"""razer""",70,100.0
"""kerry""",70,100.0
"""zappy""",70,100.0
"""koker""",70,100.0
"""waxer""",70,100.0
"""zoner""",70,100.0
"""tater""",70,100.0


# Guess word statistics

In [27]:
guess_stats = (
    first_guess
    .group_by("guess")
    .agg(
        tries_mode=pl.col("tries").mode().cast(pl.UInt8),
        tries_mean=(pl.col("tries").mean().round(3).cast(pl.Float32)),
        tries_null_pct=(pl.col("tries").is_null().mean() * 100).round(3).cast(pl.Float32),
        avg_letters_identified=(pl.col("num_common_letters").mean().round(3).cast(pl.Float32)),
        avg_letters_matched=(pl.col("num_matching_index").mean().round(3).cast(pl.Float32)),
    )
    .explode("tries_mode")
    .join(
        (
            first_guess
            .select("guess_word_anagrams", "guess")
            .unique(subset="guess", keep="first")
        ),
        left_on="guess",
        right_on="guess"
    )
)

guess_stats.head()

guess,tries_mode,tries_mean,tries_null_pct,avg_letters_identified,avg_letters_matched,guess_word_anagrams
str,u8,f32,f32,f32,f32,str
"""sarge""",4,4.44,10.09,1.618,0.581,"""sarge, sager, segar"""
"""raise""",4,4.381,10.354,1.789,0.509,"""raise, arise, serai, aesir"""
"""arose""",4,4.413,10.475,1.809,0.432,null
"""realo""",4,4.448,10.057,1.743,0.369,"""realo"""
"""sayne""",4,4.423,9.518,1.581,0.533,"""sayne"""


### Scatter Plot of Avg. letters identified vs Avg. tries

In [28]:
plot_guess_stats(
    dataf=guess_stats,
    x_axis_data="avg_letters_identified",
    y_axis_data="tries_mean",
    words="guess",
    title="Avg. letters identified vs Avg. tries",
    xaxis_title="Avg. letters identified",
    yaxis_title="Avg. tries"
)

In [ ]:
raise_anagrams = catalog.load("params:raise_anagrams")
trace_anagrams = catalog.load("params:trace_anagrams")
soare_anagrams = catalog.load("params:soare_anagrams")
salet_anagrams = catalog.load("params:salet_anagrams")

In [30]:
words_to_highlight = raise_anagrams + trace_anagrams + soare_anagrams + salet_anagrams

In [31]:

plot_guess_stats_highlighted_words(
    dataf=guess_stats,
    x_axis_data="avg_letters_identified",
    y_axis_data="tries_mean",
    words=words_to_highlight,
    title="Avg. letters identified vs Avg. tries",
    xaxis_title="Avg. letters identified",
    yaxis_title="Avg. tries"
)

In [32]:
(
    guess_stats
    .select(pl.exclude("guess_word_anagrams"))
    .filter(pl.col("guess"). is_in(soare_anagrams))
    .sort(["avg_letters_identified", "tries_mean"], descending=[True, False])
)

guess,tries_mode,tries_mean,tries_null_pct,avg_letters_identified,avg_letters_matched
str,u8,f32,f32,f32,f32
"""soare""",4,4.381,10.431,1.809,0.598
"""aeros""",4,4.407,9.87,1.809,0.461
"""arose""",4,4.413,10.475,1.809,0.432


In [33]:
(
    guess_stats
    .select(pl.exclude("guess_word_anagrams"))
    .sort(["avg_letters_identified", "tries_mean"], descending=[True, False])
    .with_row_index(offset=1)
    .head(10)
)

index,guess,tries_mode,tries_mean,tries_null_pct,avg_letters_identified,avg_letters_matched
u32,str,u8,f32,f32,f32,f32
1,"""soare""",4,4.381,10.431,1.809,0.598
2,"""aeros""",4,4.407,9.87,1.809,0.461
3,"""arose""",4,4.413,10.475,1.809,0.432
4,"""raise""",4,4.381,10.354,1.789,0.509
5,"""arise""",4,4.381,9.32,1.789,0.434
6,"""aesir""",4,4.397,9.133,1.789,0.384
7,"""serai""",4,4.399,9.837,1.789,0.472
8,"""laser""",4,4.414,9.32,1.746,0.44
9,"""realo""",4,4.448,10.057,1.743,0.369


### Scatter Plot of Avg. number of letters matched vs Avg. tries

In [34]:
plot_guess_stats(
    dataf=guess_stats,
    x_axis_data="avg_letters_matched",
    y_axis_data="tries_mean",
    words="guess",
    title="Avg. number of letters matched vs Avg. tries",
    xaxis_title="Avg. number of letters matched",
    yaxis_title="Avg. tries"
)

In [35]:
plot_guess_stats_highlighted_words(
    dataf=guess_stats,
    x_axis_data="avg_letters_matched",
    y_axis_data="tries_mean",
    words=words_to_highlight,
    title="Highlighted Scatter Plot of Avg. number of letters matched vs Avg. tries",
    xaxis_title="Avg. number of letters matched",
    yaxis_title="Avg. tries"
)

In [36]:
(
    guess_stats
    .select(["guess", "avg_letters_matched", "tries_mean"])
    .sort(["avg_letters_matched", "tries_mean"], descending=[True, False])
    .with_row_index(offset=1)
    .head(50)
)

index,guess,avg_letters_matched,tries_mean
u32,str,f32,f32
1,"""soare""",0.598,4.381
2,"""sarge""",0.581,4.44
3,"""saice""",0.576,4.379
4,"""saute""",0.568,4.425
5,"""salue""",0.56,4.415
6,"""morae""",0.559,4.425
7,"""salet""",0.556,4.357
8,"""soave""",0.554,4.522
9,"""carte""",0.551,4.395


# Challenge word statistics

In [37]:
challenge_stats = (
    simulating_top_words_tries_difficulty
    .group_by("challenge")
    .agg(
        tries_mode=pl.col("tries").mode().cast(pl.UInt8),
        tries_mean=(pl.col("tries").mean().cast(pl.Float32)).round(3),
        tries_null_pct=(pl.col("tries").is_null().mean().cast(pl.Float32) * 100).round(3),
    )
    .explode("tries_mode")
    .sort("tries_null_pct", descending = True)
)

In [38]:
print(f"Overall incompletion rate: {(first_guess.select(pl.col('tries').is_null().sum()).item() / first_guess.height) * 100:.2f}%")

Overall incompletion rate: 9.44%


In [39]:
plot_challenge_stats(
    dataf=challenge_stats,
    x_axis_data="tries_mean",
    y_axis_data="tries_null_pct",
    words="challenge",
    title="Scatter Plot of Avg. tries for Challenge words vs Failure rate",
    xaxis_title="Avg. tries",
    yaxis_title="Failure rate"
)